In [ ]:
pip install llama_index

In [ ]:
!pip install colab-env


In [ ]:
import openai
from google.colab import userdata

# Retrieve the API key from Colab Secrets
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!wget "https://moitt.gov.pk/SiteImage/Misc/files/Final%20Draft%20Personal%20Data%20Protection%20Bill%20May%202023.pdf" -O pak.pdf
!wget "https://www.paisayaar.pk/h5/privacy.html" -O paisayaar.html

--2024-12-18 02:31:30--  https://moitt.gov.pk/SiteImage/Misc/files/Final%20Draft%20Personal%20Data%20Protection%20Bill%20May%202023.pdf
Resolving moitt.gov.pk (moitt.gov.pk)... 203.101.184.86, 2407:9e00:50:1:1:1:a1:5
Connecting to moitt.gov.pk (moitt.gov.pk)|203.101.184.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660065 (645K) [application/pdf]
Saving to: ‘pak.pdf’

pak.pdf             100%[===================>] 644.59K   477KB/s    in 1.4s    

2024-12-18 02:31:34 (477 KB/s) - ‘pak.pdf’ saved [660065/660065]

--2024-12-18 02:31:34--  https://www.paisayaar.pk/h5/privacy.html
Resolving www.paisayaar.pk (www.paisayaar.pk)... 175.107.194.125
Connecting to www.paisayaar.pk (www.paisayaar.pk)|175.107.194.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36973 (36K) [text/html]
Saving to: ‘paisayaar.html’

paisayaar.html      100%[===================>]  36.11K  --.-KB/s    in 0s      

2024-12-18 02:31:36 (275 MB/s) - ‘paisayaar.ht

In [ ]:
# Install required libraries and wkhtmltopdf
!apt-get install -y wkhtmltopdf
!pip install pdfkit

import pdfkit

# Configure path to wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

# URL of the privacy policy
url = "https://www.paisayaar.pk/h5/privacy.html"

# Output PDF file name
output_file = "paisayaar_privacy_policy.pdf"

# Convert the webpage to PDF
pdfkit.from_url(url, output_file, configuration=config)

print(f"Privacy policy saved as {output_file}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Privacy policy saved as paisayaar_privacy_policy.pdf


In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from typing import List
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.vector_stores import FilterCondition

llm = OpenAI(model="gpt-4o", temperature=0)


documents = SimpleDirectoryReader(input_files=["pak.pdf"]).load_data()

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of pak.pdf"
    ),
)

# load documents
documents_2 = SimpleDirectoryReader(input_files=["paisayaar_privacy_policy.pdf"]).load_data()
splitter_2 = SentenceSplitter(chunk_size=1024)
nodes_2 = splitter_2.get_nodes_from_documents(documents_2)

vector_index = VectorStoreIndex(nodes_2)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

summary_index_2 = SummaryIndex(nodes_2)
summary_query_engine_2 = summary_index_2.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_paisayaar = QueryEngineTool.from_defaults(
    name="summary_tool_paisayaar",
    query_engine=summary_query_engine_2,
    description=(
        "Useful if you want to get a summary of paisayaar_privacy_policy.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_paisayaar],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)


In [ ]:
!pip install fpdf
!apt-get install -y fonts-dejavu


In [ ]:
from fpdf import FPDF
from google.colab import files



def generate_pdf(text, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(190, 10, txt=text)
    pdf.output(filename)
    files.download(filename)
    print(f"PDF generated and downloaded: {filename}")


In [ ]:
response = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. "
    "Review the privacy policy comprehensively and identify its alignment with these compliance requirements. "
    "Your response should include: "
    "1. A summary of key compliance obligations derived from the PDPL. "
    "2. A detailed analysis of whether the privacy policy meets these obligations. "
    "3. Any areas of non-compliance or gaps observed, with recommendations for improvement. "
    "Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis."
)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. Review the privacy policy comprehensively and identify its alignment with these compliance requirements. Your response should include: 1. A summary of key compliance obligations derived from the PDPL. 2. A detailed analysis of whether the privacy policy meets these obligations. 3. Any areas of non-compliance or gaps observed, with recommendations for improvement. Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Derive all mandatory compliance points that an application's privacy policy must address according to the Pakistan Data Protection Law (PDPL)."}
=== Function Output ===
An application's privacy policy must addres

In [ ]:
response = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. "
    "Review the privacy policy saved as paisayaar_privacy_policy.pdf comprehensively and identify its alignment with these compliance requirements. "
    "Your response should include: "
    "1. A summary of key compliance obligations derived from the PDPL. "
    "2. A detailed analysis of whether the privacy policy meets these obligations. "
    "3. Any areas of non-compliance or gaps observed, with recommendations for improvement. "
    "Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis."
)

generate_pdf(response.response, "test_prompt_2.pdf")

Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. Review the privacy policy saved as paisayaar_privacy_policy.pdf comprehensively and identify its alignment with these compliance requirements. Your response should include: 1. A summary of key compliance obligations derived from the PDPL. 2. A detailed analysis of whether the privacy policy meets these obligations. 3. Any areas of non-compliance or gaps observed, with recommendations for improvement. Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "key compliance obligations of PDPL", "page_numbers": []}
=== Function Output ===
The key compliance obligations of PDPL include ensuring robust data protection measures, addre

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: test_prompt_2.pdf


In [ ]:
Points = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address."
    "Structure it in a bullet point lis"
    "Your job is to then format and make it into a to do list against which every data protection law will be tested"
)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address.Structure it in a bullet point lisYour job is to then format and make it into a to do list against which every data protection law will be tested
=== LLM Response ===
### PDPL Compliance To-Do List for Privacy Policies

- **Grounds for Processing Personal Data**
  - [ ] Clearly define the legal basis for processing personal data.

- **Consent**
  - [ ] Obtain and document explicit consent for personal data processing.

- **Notice to Data Subject**
  - [ ] Inform data subjects about data collection and processing activities.

- **Non-Disclosure**
  - [ ] Ensure personal data is not disclosed without consent.

- **Security Requirements**
  - [ ] Implement measures to protect data from unauthorized access or breaches.

- **Data Retention**
  - [ ] Define

In [ ]:
from fpdf import FPDF
from google.colab import files

# Function to generate a PDF for a given text
def generate_pdf(text, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(190, 10, txt=text)
    pdf.output(filename)
    files.download(filename)
    print(f"PDF generated and downloaded: {filename}")

# Stage 1: Derive Compliance Points from PDPL
Stage1 = agent.chat(
    "You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). "
    "Your task is to thoroughly review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. "
    "Provide a detailed and comprehensive bullet-point list of these compliance requirements, ensuring each point is clear and actionable."
)
generate_pdf(Stage1.response, "Stage1_PDPL_Compliance_Points.pdf")

# Stage 2: Create a PDPL Compliance To-Do List
Stage2 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the compliance points derived in Stage 1, transform them into a to-do list format. "
    "This checklist should be actionable and structured in a way that each point can be used to evaluate a privacy policy's alignment with the PDPL."
)
generate_pdf(Stage2.response, "Stage2_PDPL_ToDo_List.pdf")

# Stage 3: Analyze the Privacy Policy Document
Stage3 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the to-do list created in Stage 2, analyze the 'paisayaar_privacy_policy.pdf' document. "
    "For each compliance point, provide a detailed analysis of what the privacy policy states and assess whether it aligns with the PDPL. "
    "Highlight specific clauses or sections in the privacy policy corresponding to each compliance point."
)
generate_pdf(Stage3.response, "Stage3_PDPL_Policy_Analysis.pdf")

# Stage 4: Generate the Compliance Report
Stage4 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Stage 3. "
    "Clearly state whether the privacy policy is fully compliant, partially compliant, or non-compliant with the PDPL. "
    "Provide a list of non-compliance points, along with detailed explanations of how the policy fails to meet these requirements. "
    "Include actionable recommendations to address any gaps or shortcomings in the privacy policy to ensure full compliance with the PDPL."
)
generate_pdf(Stage4.response, "Stage4_PDPL_Compliance_Report.pdf")


Added user message to memory: You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to thoroughly review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. Provide a detailed and comprehensive bullet-point list of these compliance requirements, ensuring each point is clear and actionable.
=== Calling Function ===
Calling function: summary_tool_Stage_1 with args: {"input": "2_Stage1_PDPL_Compliance_Points.pdf"}
=== Function Output ===
The document "2_Stage1_PDPL_Compliance_Points.pdf" outlines detailed compliance requirements for privacy policies under the Pakistan Data Protection Law (PDPL). It covers aspects such as data collection practices, user consent mechanisms, user rights including access, correction, and erasure, data sharing with third parties, data retention and deletion policies, data security measures, international data transfers, policy upd

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage1_PDPL_Compliance_Points.pdf
Added user message to memory: You are an expert in data protection laws and compliance. Using the compliance points derived in Stage 1, transform them into a to-do list format. This checklist should be actionable and structured in a way that each point can be used to evaluate a privacy policy's alignment with the PDPL.
=== Calling Function ===
Calling function: summary_tool_Stage_2 with args: {"input": "2_Stage2_PDPL_ToDo_List.pdf"}
=== Function Output ===
The provided information outlines a structured and actionable compliance to-do list based on the PDPL requirements. It includes tasks categorized under Data Collection and Processing, User Rights, Data Sharing and Security, Data Retention and Deletion, International Data Transfers, Policy Updates and User Notification, and Adherence to Specific Terms. Each task specifies actions to be taken to ensure compliance with the PDPL, along with references to specific clauses, pa

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage3_PDPL_Policy_Analysis.pdf
Added user message to memory: You are an expert in data protection laws and compliance. Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Stage 3. Clearly state whether the privacy policy is fully compliant, partially compliant, or non-compliant with the PDPL. Provide a list of non-compliance points, along with detailed explanations of how the policy fails to meet these requirements. Include actionable recommendations to address any gaps or shortcomings in the privacy policy to ensure full compliance with the PDPL.
=== LLM Response ===
Based on the analysis performed in Stage 3, the 'paisayaar_privacy_policy.pdf' document is **fully compliant** with the Pakistan Data Protection Law (PDPL). The analysis indicates that the privacy policy aligns with all the key requirements of the PDPL, ensuring robust data protection and user privacy. 

### Compliance Status
- **

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage4_PDPL_Compliance_Report.pdf


In [ ]:
Stage1 = agent.chat(
    "You are an expert in data protection laws and compliance, with a specialization in the Pakistan Data Protection Law (PDPL). "
    "Carefully review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. "
    "Provide a clear, detailed, and comprehensive bullet-point list of these compliance requirements. "
    "Ensure that each compliance point is actionable and unambiguous, covering key areas such as: "
    "- Data collection practices "
    "- User consent mechanisms "
    "- User rights (e.g., access, correction, erasure) "
    "- Data sharing with third parties "
    "- Data retention and deletion policies "
    "- Data security measures "
    "- International data transfers "
    "Include specific references to the clause, page, and line number in the PDPL for each compliance requirement."
)

generate_pdf(Stage1.response, "2_Stage1_PDPL_Compliance_Points.pdf")


Added user message to memory: You are an expert in data protection laws and compliance, with a specialization in the Pakistan Data Protection Law (PDPL). Carefully review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. Provide a clear, detailed, and comprehensive bullet-point list of these compliance requirements. Ensure that each compliance point is actionable and unambiguous, covering key areas such as: - Data collection practices - User consent mechanisms - User rights (e.g., access, correction, erasure) - Data sharing with third parties - Data retention and deletion policies - Data security measures - International data transfers Include specific references to the clause, page, and line number in the PDPL for each compliance requirement.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "mandatory compliance requirements for privacy policy under PDPL", "page_numbers": []}
=== Function Output ===

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage1_PDPL_Compliance_Points.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage1_PDPL_Compliance_Points.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_1 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_1",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage1_PDPL_Compliance_Points.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_Stage_1],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage2 = agent.chat(
    "Using the compliance points derived in Stage 1 (Saved in the pdf '2_Stage1_PDPL_Compliance_Points.pdf'), create a structured and actionable compliance to-do list. "
    "This checklist should be organized into categories that align with the PDPL requirements (e.g., data collection, user consent, security, retention). "
    "Each task in the to-do list should be concise, testable, and include a reference to the relevant PDPL clause, page, and line number. "
    "This to-do list will serve as a framework for evaluating privacy policies for compliance with the PDPL."
)

generate_pdf(Stage2.response, "2_Stage2_PDPL_ToDo_List.pdf")


Added user message to memory: Using the compliance points derived in Stage 1 (Saved in the pdf '2_Stage1_PDPL_Compliance_Points.pdf'), create a structured and actionable compliance to-do list. This checklist should be organized into categories that align with the PDPL requirements (e.g., data collection, user consent, security, retention). Each task in the to-do list should be concise, testable, and include a reference to the relevant PDPL clause, page, and line number. This to-do list will serve as a framework for evaluating privacy policies for compliance with the PDPL.
=== Calling Function ===
Calling function: summary_tool_Stage_1 with args: {"input": "Create a structured and actionable compliance to-do list based on the compliance points derived in Stage 1, organized into categories that align with the PDPL requirements."}
=== Function Output ===
Create a structured and actionable compliance to-do list based on the compliance points derived in Stage 1, organized into categories th

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage2_PDPL_ToDo_List.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage2_PDPL_ToDo_List.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_2 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_2",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage2_PDPL_ToDo_List.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_Stage_1, summary_tool_Stage_2],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage3 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the PDPL compliance to-do list created in Stage 2 (saved as 2_Stage2_PDPL_ToDo_List.pdf), analyze the 'paisayaar_privacy_policy.pdf' document. "
    "For each compliance point, provide: "
    "- A detailed explanation of what the privacy policy states about that specific point. "
    "- An assessment of whether the privacy policy meets the PDPL requirement. "
    "- Specific references to the clause, page, and line number in the PDPL that the compliance point corresponds to. "
    "- Highlight any areas of non-compliance and explain the gaps in detail. "
    "Ensure that all references to PDPL clauses are accurate and specific."
)

generate_pdf(Stage3.response, "2_Stage3_PDPL_Policy_Analysis.pdf")


Added user message to memory: You are an expert in data protection laws and compliance. Using the PDPL compliance to-do list created in Stage 2 (saved as 2_Stage2_PDPL_ToDo_List.pdf), analyze the 'paisayaar_privacy_policy.pdf' document. For each compliance point, provide: - A detailed explanation of what the privacy policy states about that specific point. - An assessment of whether the privacy policy meets the PDPL requirement. - Specific references to the clause, page, and line number in the PDPL that the compliance point corresponds to. - Highlight any areas of non-compliance and explain the gaps in detail. Ensure that all references to PDPL clauses are accurate and specific.
=== Calling Function ===
Calling function: summary_tool_Stage_2 with args: {"input": "Summarize the PDPL compliance to-do list from Stage 2."}
=== Function Output ===
The PDPL compliance to-do list from Stage 2 includes tasks related to Data Security Measures, Data Retention and Deletion, International Data Tra

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage3_PDPL_Policy_Analysis.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage3_PDPL_Policy_Analysis.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)



summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_3 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_3",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage3_PDPL_Policy_Analysis.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage4 = agent.chat(
    "Generate a comprehensive compliance report based on the analysis performed in Stage 3 (saved as 2_Stage3_PDPL_Policy_Analysis.pdf) of the 'paisayaar_privacy_policy.pdf' document. "
    "The report must include: "
    "1. A clear summary of the compliance status, indicating whether the privacy policy is fully compliant, partially compliant, or non-compliant with the Pakistan Data Protection Law (PDPL). "
    "2. A detailed list of non-compliance points, including: "
    "   - A description of how the policy fails to meet the PDPL requirement. "
    "   - References to the specific PDPL clause, page, and line number being violated. "
    "3. Actionable recommendations for addressing each area of non-compliance to achieve full alignment with the PDPL. "
    "Structure the report clearly with headings and bullet points for easy readability."
)



generate_pdf(Stage4.response, "2_Stage4_PDPL_Compliance_Report.pdf")


Added user message to memory: Generate a comprehensive compliance report based on the analysis performed in Stage 3 (saved as 2_Stage3_PDPL_Policy_Analysis.pdf) of the 'paisayaar_privacy_policy.pdf' document. The report must include: 1. A clear summary of the compliance status, indicating whether the privacy policy is fully compliant, partially compliant, or non-compliant with the Pakistan Data Protection Law (PDPL). 2. A detailed list of non-compliance points, including:    - A description of how the policy fails to meet the PDPL requirement.    - References to the specific PDPL clause, page, and line number being violated. 3. Actionable recommendations for addressing each area of non-compliance to achieve full alignment with the PDPL. Structure the report clearly with headings and bullet points for easy readability.
=== Calling Function ===
Calling function: summary_tool_Stage_3 with args: {"input": "Generate a comprehensive compliance report based on the analysis performed in Stage 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage4_PDPL_Compliance_Report.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage4_PDPL_Compliance_Report.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_4 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_4",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage4_PDPL_Compliance_Report.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3, summary_tool_Stage_4],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage5 = agent.chat(
    "You are an expert in compliance validation with a specialization in data protection laws, particularly the Pakistan Data Protection Law (PDPL). "
    "Your task is to thoroughly validate the compliance report generated in Stage 4 (saved as '2_Stage4_PDPL_Compliance_Report.pdf') to ensure its accuracy, consistency, and reliability. "
    "Follow the steps below and provide a structured, clear, and actionable validation output: "

    "### Step 1: Cross-Check PDPL References ### "
    "For each compliance point and non-compliance point mentioned in the report: "
    "1. Verify whether the referenced PDPL clause, page, and line number actually exist in the PDPL document. "
    "2. Quote the exact wording of the relevant clause from the PDPL. "
    "3. Compare this clause with the corresponding section or wording in the privacy policy document. Quote the exact wording from the privacy policy for clarity. "
    "4. Provide a structured comparison: "
    "   - Compliance Point: [Describe the compliance point from the report] "
    "   - PDPL Requirement (Quoted): [Quote the exact PDPL clause] "
    "   - Privacy Policy Content (Quoted): [Quote what the privacy policy says] "
    "   - Assessment: [State whether the compliance/non-compliance determination is accurate based on this comparison] "

    "### Step 2: Validate the Assessment ### "
    "Carefully review whether the report's assessment of compliance or non-compliance is accurate. "
    "Confirm that the reasoning aligns with both the PDPL requirements and the privacy policy's content. Highlight any discrepancies found."

    "### Step 3: Evaluate Recommendations ### "
    "Assess the recommendations provided in the compliance report: "
    "1. Verify whether the recommendations are actionable, clear, and directly address the non-compliance or gaps identified. "
    "2. Suggest improvements to any recommendations that are unclear or insufficient for achieving full compliance. "

    "### Step 4: Validation Summary ### "
    "Summarize the overall validation results, providing: "
    "1. A clear statement on whether the compliance report is accurate, consistent, and complete. "
    "2. A list of any errors, inaccuracies, or missing references found in the report. "
    "3. Suggested corrections or improvements to ensure the report meets the highest standard of accuracy and reliability. "

    "### Format Your Response ### "
    "Structure your response into the following sections: "
    "1. Validation of PDPL References "
    "2. Validation of Compliance Assessments "
    "3. Evaluation of Recommendations "
    "4. Final Validation Summary "

    "Be thorough, precise, and clear in your analysis. Use structured bullet points and headings for readability, ensuring that the final output can serve as a reliable validation of the compliance report."
)


generate_pdf(Stage4.response, "2_Stage5_PDPL_Compliance_Report.pdf")


Added user message to memory: You are an expert in compliance validation with a specialization in data protection laws, particularly the Pakistan Data Protection Law (PDPL). Your task is to thoroughly validate the compliance report generated in Stage 4 (saved as '2_Stage4_PDPL_Compliance_Report.pdf') to ensure its accuracy, consistency, and reliability. Follow the steps below and provide a structured, clear, and actionable validation output: ### Step 1: Cross-Check PDPL References ### For each compliance point and non-compliance point mentioned in the report: 1. Verify whether the referenced PDPL clause, page, and line number actually exist in the PDPL document. 2. Quote the exact wording of the relevant clause from the PDPL. 3. Compare this clause with the corresponding section or wording in the privacy policy document. Quote the exact wording from the privacy policy for clarity. 4. Provide a structured comparison:    - Compliance Point: [Describe the compliance point from the report]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage5_PDPL_Compliance_Report.pdf


In [ ]:
# response = agent.chat(
#     "You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to assess the compliance of the 'paisayaar_privacy_policy.pdf' document with the PDPL. Follow these steps systematically and produce a structured output for each: "



#     "### Step 1: Derive Compliance Points from PDPL ### "
#     "Carefully analyze the PDPL to extract all mandatory compliance requirements that an application's privacy policy must address. "
#     "Provide a detailed and actionable bullet-point list covering key areas such as: "
#     "- Data collection practices "
#     "- User consent mechanisms "
#     "- User rights (e.g., access, correction, erasure) "
#     "- Data sharing with third parties "
#     "- Data retention and deletion policies "
#     "- Data security measures "
#     "- International data transfers "
#     "Include specific references to the clause, page, and line number from the PDPL for each compliance point. "

#     "### Step 2: Create a PDPL Compliance To-Do List ### "
#     "Based on the compliance points derived in Step 1, create a structured and actionable compliance to-do list. "
#     "Organize it into categories (e.g., data collection, user consent, security, retention) with concise, testable tasks. "
#     "Ensure each item includes a reference to the relevant clause, page, and line number in the PDPL. "

#     "### Step 3: Analyze the Privacy Policy Document ### "
#     "Using the compliance to-do list from Step 2, evaluate the 'paisayaar_privacy_policy.pdf' document. For each compliance point: "
#     "- Provide a detailed explanation of what the privacy policy states about that point. "
#     "- Assess whether the privacy policy meets the PDPL requirement. "
#     "- Reference specific clauses or sections in the privacy policy relevant to each compliance point. "
#     "- Highlight non-compliance areas with detailed explanations and include references to the relevant clause, page, and line number in the PDPL. "

#     "### Step 4: Generate the Compliance Report ### "
#     "Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Step 3. "
#     "The report must include: "
#     "1. A clear summary of the compliance status: fully compliant, partially compliant, or non-compliant. "
#     "2. A detailed list of non-compliance points, explaining how the policy fails to meet PDPL requirements, with references to the specific clause, page, and line number. "
#     "3. Actionable recommendations for addressing each area of non-compliance to achieve full PDPL compliance. "
#     "Structure the report clearly with headings and bullet points for easy readability. "

#     "### Step 5: Validate the Compliance Report ### "
#     "Carefully validate the compliance report generated in Step 4. "
#     "For each compliance and non-compliance point, cross-check the findings with the PDPL document to ensure accuracy and consistency. "
#     "Confirm that the referenced clauses, pages, and lines are correct, and verify the alignment of the assessment with the privacy policy content. "
#     "Provide a validation summary stating whether the report is accurate and complete. If any corrections or improvements are needed, include those as well."
# )

# generate_pdf(Stage4.response, "Run_Poc.pdf")



Added user message to memory: You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to assess the compliance of the 'paisayaar_privacy_policy.pdf' document with the PDPL. Follow these steps systematically and produce a structured output for each: ### Step 1: Derive Compliance Points from PDPL ### Carefully analyze the PDPL to extract all mandatory compliance requirements that an application's privacy policy must address. Provide a detailed and actionable bullet-point list covering key areas such as: - Data collection practices - User consent mechanisms - User rights (e.g., access, correction, erasure) - Data sharing with third parties - Data retention and deletion policies - Data security measures - International data transfers Include specific references to the clause, page, and line number from the PDPL for each compliance point. ### Step 2: Create a PDPL Compliance To-Do List ### Based on the compliance points

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Run_Poc.pdf


In [ ]:
from google.colab import files


test = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully review the Pakistan Data Protection Law (PDPL) and derive a comprehensive checklist of all mandatory compliance points that an application's privacy policy must address. "
    "The checklist should be structured, clear, and actionable, covering all relevant compliance areas based on the PDPL. "
    "Focus on identifying key requirements and organizing them into a format that can be used to evaluate privacy policy compliance. "
)


generate_pdf(test.response, "test_prompt_1.pdf")






Added user message to memory: You are an expert in data protection laws and compliance. Carefully review the Pakistan Data Protection Law (PDPL) and derive a comprehensive checklist of all mandatory compliance points that an application's privacy policy must address. The checklist should be structured, clear, and actionable, covering all relevant compliance areas based on the PDPL. Focus on identifying key requirements and organizing them into a format that can be used to evaluate privacy policy compliance. 
=== Calling Function ===
Calling function: vector_tool with args: {"query": "compliance checklist for privacy policy based on Pakistan Data Protection Law (PDPL)", "page_numbers": []}
=== Function Output ===
The compliance checklist for a privacy policy based on the Pakistan Data Protection Law (PDPL) should include the following key points:
1. Ensure that the privacy policy effectively addresses all key areas required by the PDPL.
2. Conduct periodic reviews of the privacy policy 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: test_prompt_1.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["test_prompt_1.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_test_prompt_1 = QueryEngineTool.from_defaults(
    name="summary_tool_test_prompt_1",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of test_prompt_1.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3, summary_tool_Stage_4, summary_tool_test_prompt_1],
    llm=llm,
     system_prompt=""" \
You are an expert in data protection laws and compliance.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
test = agent.chat(
    "Using the summary provided by 'summary_tool_test_prompt_1' (saved as 'test_prompt_1.pdf'), evaluate the compliance of 'paisayaar_privacy_policy.pdf' with the Pakistan Data Protection Law (PDPL) outlined in 'pak.pdf'. "
    "Generate a well-structured report that clearly lists: "
    "1. Compliance points where Paisayaar's privacy policy aligns with the PDPL. "
    "2. Non-compliance points, including specific areas requiring improvement. "
    "Ensure the report is clear, organized, and actionable."
    "Recheck in the end the results shouldnt be hallucinated"
)


Added user message to memory: Using the summary provided by 'summary_tool_test_prompt_1' (saved as 'test_prompt_1.pdf'), evaluate the compliance of 'paisayaar_privacy_policy.pdf' with the Pakistan Data Protection Law (PDPL) outlined in 'pak.pdf'. Generate a well-structured report that clearly lists: 1. Compliance points where Paisayaar's privacy policy aligns with the PDPL. 2. Non-compliance points, including specific areas requiring improvement. Ensure the report is clear, organized, and actionable.Recheck in the end the results shouldnt be hallucinated
=== Calling Function ===
Calling function: summary_tool_test_prompt_1 with args: {"input": "test_prompt_1.pdf"}
=== Function Output ===
The Paisayaar privacy policy has been evaluated for compliance with the Pakistan Data Protection Law (PDPL) and has been found to be fully compliant. The policy aligns with various aspects of the PDPL, including lawful processing and consent, data subject notification, security measures, data retention

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: test_prompt_1.pdf


In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3, summary_tool_Stage_4, summary_tool_test_prompt_1],
    llm=llm,
     system_prompt=""" \
You are a professional in data protection laws and compliance. Your job is to rigorously test privay policy of apps and find areas where it is not compliant.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
""",
    verbose=True
)
agent = AgentRunner(agent_worker)

# test = agent.chat(
#     "You are an expert in data protection laws and compliance. "
#     "Conduct a rigorous and thorough evaluation of 'paisayaar_privacy_policy.pdf' to determine its compliance with the Pakistan Data Protection Law (PDPL) detailed in 'pak.pdf'. "
#     "Use the summary provided in 'summary_tool_test_prompt_1' (saved as 'test_prompt_1.pdf') as a reference. "
#     "Identify: "
#     "1. Points where the privacy policy complies with the PDPL. "
#     "2. Points where it does not comply, with detailed explanations of the gaps or shortcomings. "
#     "3. Areas that need improvement to achieve full compliance. "
#     "Ensure the output is comprehensive, well-structured, and actionable, providing clear evidence and references for each finding."
# )

generate_pdf(test.response, "rigorous_prompt_2.pdf")


Added user message to memory: You are an expert in data protection laws and compliance. Conduct a rigorous and thorough evaluation of 'paisayaar_privacy_policy.pdf' to determine its compliance with the Pakistan Data Protection Law (PDPL) detailed in 'pak.pdf'. Use the summary provided in 'summary_tool_test_prompt_1' (saved as 'test_prompt_1.pdf') as a reference. Identify: 1. Points where the privacy policy complies with the PDPL. 2. Points where it does not comply, with detailed explanations of the gaps or shortcomings. 3. Areas that need improvement to achieve full compliance. Ensure the output is comprehensive, well-structured, and actionable, providing clear evidence and references for each finding.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "pak.pdf"}
=== Function Output ===
The document "pak.pdf" provides detailed information about the establishment, functions, powers, and regulations of a Commission responsible for data protection in Pakistan. It

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: rigorous_prompt_2.pdf
